In [5]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import pandas as pd
import pickle
import random

In [6]:
# import our chat-bot intents file
import json

with open('intents.json') as json_data:
    intents = json.load(json_data)

words = []
classes = []
documents = []
ignore_words = ['?','!']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique stemmed words", words)

62 documents
15 classes ['check_ahu', 'check_all_hvac', 'check_boiler_plant', 'check_chiller_plant', 'check_energy', 'check_future_weather', 'check_how_it_should_all_work', 'check_power', 'check_runtime', 'check_zone_temps', 'goodbye', 'greeting', 'options', 'thanks', 'what_to_do']
152 unique stemmed words ["'s", ',', 'a', 'ahu', 'air', 'al', 'alarm', 'algorithm', 'an', 'and', 'any', 'anyon', 'ar', 'area', 'at', 'awesom', 'bad', 'be', 'been', 'big', 'boil', 'box', 'build', 'bye', 'cal', 'can', 'cent', 'chat', 'chil', 'cold', 'consum', 'control', 'cool', 'could', 'damp', 'day', 'did', 'discharg', 'do', 'doe', 'doing', 'down', 'driv', 'duc', 'dud', 'econom', 'elect', 'energy', 'equip', 'fan', 'fir', 'for', 'fre', 'freez', 'frequ', 'from', 'front', 'fuel', 'get', 'going', 'good', 'goodby', 'handl', 'has', 'hello', 'help', 'hi', 'hot', 'hour', 'how', 'hvac', 'i', 'ilc', 'in', 'intellig', 'is', 'kw', 'kwh', 'lat', 'leav', 'lik', 'load', 'log', 'look', 'mak', 'many', 'me', 'mix', 'mod', 'mor

In [3]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # stem each word - create base word, in attempt to represent related words
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

<ipython-input-3-72b6ba6ac079>:30: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


In [4]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
modelo = Sequential()
modelo.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(64, activation='relu'))
modelo.add(Dropout(0.5))
modelo.add(Dense(len(train_y[0]), activation='softmax'))

In [5]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
modelo.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [6]:
# Fit the model
modelo.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
13/13 [==============================] - 1s 1ms/step - loss: 2.7130 - accuracy: 0.1338
Epoch 2/200
13/13 [==============================] - 0s 916us/step - loss: 2.5846 - accuracy: 0.1199
Epoch 3/200
13/13 [==============================] - 0s 1ms/step - loss: 2.5540 - accuracy: 0.1312
Epoch 4/200
13/13 [==============================] - 0s 1ms/step - loss: 2.3822 - accuracy: 0.2966
Epoch 5/200
13/13 [==============================] - 0s 1ms/step - loss: 2.2859 - accuracy: 0.2921
Epoch 6/200
13/13 [==============================] - 0s 1ms/step - loss: 2.0712 - accuracy: 0.3400
Epoch 7/200
13/13 [==============================] - 0s 2ms/step - loss: 2.0551 - accuracy: 0.5109
Epoch 8/200
13/13 [==============================] - 0s 1ms/step - loss: 1.8429 - accuracy: 0.5303
Epoch 9/200
13/13 [==============================] - 0s 1ms/step - loss: 1.8452 - accuracy: 0.3635
Epoch 10/200
13/13 [==============================] - 0s 1ms/step - loss: 1.6897 - accuracy: 0.4335
Epoch 1

In [7]:
modelo.save('saved_model/chatbot-brains-model')

INFO:tensorflow:Assets written to: saved_model/chatbot-brains-model\assets


In [7]:
import tensorflow as tf

loaded_model = tf.keras.models.load_model('saved_model/chatbot-brains-model')

In [8]:


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
    

def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    # generate probabilities from the model
    input_data = pd.DataFrame([bow(sentence, words)], dtype=float, index=['input'])
    results = loaded_model.predict([input_data])[0]
    # filter out predictions below a threshold, and provide intent index
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    # return tuple of intent and probability
    
    return return_list

In [9]:
p = bow("HVAC is a mechanical system in a building I think", words)

print(p)
print(classes)

found in bag: hvac
found in bag: is
found in bag: a
found in bag: system
found in bag: in
found in bag: a
found in bag: build
found in bag: i
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0]
['check_ahu', 'check_all_hvac', 'check_boiler_plant', 'check_chiller_plant', 'check_energy', 'check_future_weather', 'check_how_it_should_all_work', 'check_power', 'check_runtime', 'check_zone_temps', 'goodbye', 'greeting', 'options', 'thanks', 'what_to_do']


In [10]:
inputvar = pd.DataFrame([p], dtype=float, index=['input'])

print(loaded_model.predict(inputvar))

[[3.7224705e-03 9.8246711e-01 8.3865807e-06 2.4179151e-05 5.2356854e-06
  9.9426666e-03 7.1380215e-05 3.4981950e-03 1.2194248e-05 1.3288302e-06
  6.3486455e-05 8.2605788e-05 1.8847154e-06 5.9693352e-06 9.2883718e-05]]


In [11]:
classify_local('Hello, good day!')

found in bag: hello
found in bag: ,
found in bag: good
found in bag: day


[('greeting', '0.9999951')]

In [12]:
classify_local('How you can assist me?')

found in bag: how
found in bag: you
found in bag: can
found in bag: me


[('options', '0.9999615')]

In [13]:
classify_local('what is my mechanical system doing?')

found in bag: what
found in bag: is
found in bag: my
found in bag: system
found in bag: doing


[('check_all_hvac', '0.6285704'), ('check_ahu', '0.36847687')]

In [14]:
classify_local('are people complaining at all?')

found in bag: ar
found in bag: at
found in bag: al


[('greeting', '0.5450222'), ('check_chiller_plant', '0.26023936')]

In [15]:
classify_local('how is the HVAC running?')

found in bag: how
found in bag: is
found in bag: the
found in bag: hvac
found in bag: run


[('check_all_hvac', '0.99990964')]

In [16]:
classify_local('has the power use for the building been bad?')

found in bag: has
found in bag: the
found in bag: pow
found in bag: us
found in bag: for
found in bag: the
found in bag: build
found in bag: been
found in bag: bad


[('check_power', '0.99926144')]

In [17]:
classify_local('are we getting any free cooling or using mechanical cooling?')

found in bag: ar
found in bag: we
found in bag: get
found in bag: any
found in bag: fre
found in bag: cool
found in bag: or
found in bag: us
found in bag: cool


[('check_chiller_plant', '0.98735654')]

In [18]:
classify_local('is the fan systems out on alarm?')

found in bag: is
found in bag: the
found in bag: fan
found in bag: system
found in bag: out
found in bag: on
found in bag: alarm


[('check_ahu', '0.99999976')]

In [19]:
classify_local('does that seem like its working correctly?')

found in bag: doe
found in bag: that
found in bag: lik
found in bag: work


[('what_to_do', '0.34076178'), ('check_ahu', '0.27212602')]

In [20]:
classify_local('are any of the vav boxes not moving air?')

found in bag: ar
found in bag: any
found in bag: the
found in bag: vav
found in bag: box
found in bag: mov
found in bag: air


[('check_zone_temps', '0.98340446')]

In [21]:
classify_local('what are the discharge temps looking like on the reheat vav box coils?')

found in bag: what
found in bag: ar
found in bag: the
found in bag: discharg
found in bag: temp
found in bag: look
found in bag: lik
found in bag: on
found in bag: the
found in bag: reh
found in bag: vav
found in bag: box


[('check_zone_temps', '0.9999263')]